In [ ]:
!pip install transformers==4.46.2 datasets==3.1.0 torch==2.5.1+cu121 sklearn-pandas==2.2.0 numpy==1.26.4

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
import pandas as pd
import datasets
import torch
import time
import os

The WANDB_API_KEY is nessesary to use the Trainer class from HuggingFace and is what's used in the example notebook. However, I think we may all have to put in our own api key secret into google colab for it to work.

In [ ]:
from google.colab import userdata
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
langs = ['java', 'python', 'pharo']

# langs = ['java'] # Using Java as the only language for testing purposes.
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

training = True

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [ ]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [ ]:
model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [ ]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
    return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [ ]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float16")))
  processed_dataset.set_format('pt')
  return processed_dataset

Evaluating the predictions.

Below is non-functional code I still haven't finnished. We need to implement a way for the logits returned from the model to be converted into binary (0 - 1) predictions for the labels, and for those labels to be compaired to the ground truth labels of the testing datasets for each language. The competition also wants us to complete this step using profiling that they have set up, and which is partially shown below. The link to the reference notebook for the competition is [here](https://colab.research.google.com/drive/1RULzasjO_nrqiXLrGze-PznFlHKtGQ4s?usp=sharing). Also, another notebook that is fairly close to what I believe we are trying to do can be found [here](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb).

In [ ]:
# scores must be global so that it can later be referenced for printing the results
scores = pd.DataFrame(columns=['lan', 'cat', 'precision', 'recall', 'f1'])

In [ ]:
def compute_metrics(eval_pred, lang, categories):
    global scores
    scores.drop(scores[scores['lan'] == lang].index, inplace=True)

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame(metrics_list)
    scores = pd.concat([scores, temp_scores], ignore_index=True)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [ ]:
def compute_metrics_validate(eval_pred, lang, categories):

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame(metrics_list)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [ ]:
def measure_runtime_and_flops(trainer, test_dataset):
  print(f' \n** Evaluating runtime and flops for {lang} test set. **\n')
  total_time = 0
  total_flops = 0

  for i in range(10):
    for batch in test_dataset:
        inputs = {key: val.unsqueeze(0).to(trainer.model.device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
        with torch.profiler.profile(with_flops=True, activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
            start_time = time.time()
            _ = trainer.model(**inputs)
            end_time = time.time()

    total_time += (end_time - start_time)
    total_flops += sum(k.flops for k in prof.key_averages()) / 1e9

    avg_runtime = total_time / 10
    avg_flops = total_flops / 10
    return avg_runtime, avg_flops, total_flops, total_time

In [ ]:
# Wrapper is needed because model can only take eval_pred for compute_metrics, but lang and labels are necessary for output
def compute_metrics_wrapper(eval_pred):
  if training:
    return compute_metrics_validate(eval_pred, lang, labels[lang])
  else:
    print(f'\n ** Evaluating metrics for {lang} test set. ** \n')
    return compute_metrics(eval_pred, lang, labels[lang])


In [ ]:
hyperparams = {
    'java': {
        'epochs': 20,
        'lr': 5e-5,
        'weight_decay': 0.01,
        'batch_size': 4
    },
    'python': {
        'epochs': 20,
        'lr': 5e-5,
        'weight_decay': 0.01,
        'batch_size': 4
    },
    'pharo': {
        'epochs': 20,
        'lr': 5e-5,
        'weight_decay': 0.01,
        'batch_size': 4
    }
}

In [ ]:
total_flops = 0
total_time = 0
metrics_dict = {}

# Main Training Loop
for lang in langs:

    training = True

    num_labels = len(labels[lang])
    model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                               num_labels=num_labels,
                                                               problem_type="multi_label_classification")

    dataset = preprocess_dataset(ds[f'{lang}_train'])
    test_dataset = preprocess_dataset(ds[f'{lang}_test'])

    train_val_split = dataset.train_test_split(test_size=0.2, seed=27)
    train_dataset = train_val_split['train']
    validation_dataset = train_val_split['test']

    training_args = TrainingArguments(
        output_dir=f'./results_{lang}',
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f'./logs_{lang}',
        per_device_train_batch_size=hyperparams[lang]['batch_size'],
        per_device_eval_batch_size=hyperparams[lang]['batch_size'],
        num_train_epochs=hyperparams[lang]['epochs'],
        weight_decay=hyperparams[lang]['weight_decay'],
        learning_rate=hyperparams[lang]['lr'],
        logging_steps=10,
        save_total_limit=1,
        load_best_model_at_end=False,
        # metric_for_best_model="f1",
        seed=27
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        processing_class=tokenizer,
        compute_metrics=compute_metrics_wrapper,
    )

    print(f'\n** Training {lang} model. **\n')
    trainer.train()
    trainer.evaluate()

    training = False

    test_trainer = Trainer(
        model=trainer.model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        processing_class=tokenizer,
        compute_metrics=compute_metrics_wrapper,
    )

    test_trainer.evaluate()

    avg_runtime, avg_flops, total_flops, total_time = measure_runtime_and_flops(test_trainer, test_dataset)

    metrics_dict[lang] = {
        'avg_runtime': avg_runtime,
        'avg_flops': avg_flops,
        'total_flops': total_flops,
        'total_time': total_time
    }

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7614 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.



** Training java model. **



...

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/289 [00:00<?, ? examples/s]


** Training pharo model. **



Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.241300,0.284376,0.253598,0.353231,0.239719
2,0.335800,0.230302,0.496358,0.613520,0.431584
3,0.147300,0.220002,0.515885,0.608157,0.461842
4,0.077900,0.234778,0.643444,0.748190,0.577745
5,0.170300,0.266532,0.583370,0.795037,0.535950
6,0.063700,0.273863,0.605046,0.782860,0.556949
7,0.041100,0.289159,0.655544,0.815473,0.602566
8,0.018600,0.311518,0.647999,0.772085,0.619038
9,0.015700,0.332768,0.658993,0.793627,0.617781
10,0.007200,0.309455,0.680310,0.819783,0.620219



 ** Evaluating metrics for pharo test set. ** 

 
** Evaluating runtime and flops for pharo test set. **



In [ ]:
scores

,lan,cat,precision,recall,f1
0,java,summary,0.877155,0.912556,0.894505
1,java,Ownership,1.000000,1.000000,1.000000
2,java,Expand,0.493333,0.362745,0.418079
3,java,usage,0.934037,0.821346,0.874074
4,java,Pointer,0.790179,0.961957,0.867647
5,java,deprecation,1.000000,0.600000,0.750000
6,java,rational,0.317460,0.294118,0.305344
7,python,Usage,0.827957,0.636364,0.719626
8,python,Parameters,0.819672,0.781250,0.800000
9,python,DevelopmentNotes,0.375000,0.365854,0.370370


In [ ]:
try:
  scores.to_csv(f'/content/drive/MyDrive/CS_440_Experiment_Results/distilbert_scores_5.5.csv')
  print('Saved scores.')
except:
  print('Could not save scores.')

Saved scores.


In [ ]:
for lang in metrics_dict:
  print(f'{lang}: {metrics_dict[lang]}')

java: {'avg_runtime': 0.0010368108749389648, 'avg_flops': 1.087410432, 'total_flops': 10.87410432, 'total_time': 0.010368108749389648}
python: {'avg_runtime': 0.0010786056518554688, 'avg_flops': 1.0874101248, 'total_flops': 10.874101248, 'total_time': 0.010786056518554688}
pharo: {'avg_runtime': 0.0009846687316894531, 'avg_flops': 1.087410432, 'total_flops': 10.87410432, 'total_time': 0.009846687316894531}


In [ ]:
total_flops = 0
total_time = 0
avg_runtime = 0
avg_flops = 0

for lang in metrics_dict:
  total_flops += metrics_dict[lang]['total_flops']
  total_time += metrics_dict[lang]['total_time']
  avg_runtime += metrics_dict[lang]['avg_runtime']
  avg_flops += metrics_dict[lang]['avg_flops']

In [ ]:
print(f'Total Flops: {total_flops}')
print(f'Total Time: {total_time}')
print(f'Average Flops: {avg_flops}')
print(f'Average Time: {avg_runtime}')

print()
for lang in langs:
  f1_ave_lang = scores[scores['lan'] == lang]['f1'].mean()
  print(f'Average F1 for {lang}: {f1_ave_lang}')

  precision_ave_lang = scores[scores['lan'] == lang]['precision'].mean()
  print(f'Average Precision for {lang}: {precision_ave_lang}')

  recall_ave_lang = scores[scores['lan'] == lang]['recall'].mean()
  print(f'Average Recall for {lang}: {recall_ave_lang}')
  print()

print()
f1_mean = scores['f1'].mean()
print(f'Average F1 for across all languages: {f1_mean}')
precision_mean = scores['precision'].mean()
print(f'Average Precision for across all languages: {precision_mean}')
recall_mean = scores['recall'].mean()
print(f'Average Recall for across all languages: {recall_mean}')

Total Flops: 32.622309888000004
Total Time: 0.031000852584838867
Average Flops: 3.2622309888000003
Average Time: 0.003100085258483887

Average F1 for java: 0.729949890699811
Average Precision for java: 0.7731663334214284
Average Recall for java: 0.7075315754722059

Average F1 for python: 0.6087431093718265
Average Precision for python: 0.6401208357044198
Average Recall for python: 0.5832178492239468

Average F1 for pharo: 0.6351315691560914
Average Precision for pharo: 0.6836993145391584
Average Recall for pharo: 0.6092591956281618


Average F1 for across all languages: 0.6631203034658131
Average Precision for across all languages: 0.7051928270655899
Average Recall for across all languages: 0.6386118233590689


In [ ]:
max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = scores['f1'].mean()
print(f'Average F1: {avg_f1}')

print(f'score: {round(score(avg_f1, avg_runtime, avg_flops), 2)}')

Average F1: 0.6631203034658131
score: 0.8
